In [1]:
from tkinter import *
from tkinter import font
from tkinter.filedialog import askopenfile
from typing import MutableSet
from tkVideoPlayer import TkinterVideo
import tensorflow as tf
from glob import glob
import os
import cv2
import numpy as np
from tensorflow.keras.utils import CustomObjectScope
H = 512
W = 512

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
    model = tf.keras.models.load_model("model_1.h5")

window=Tk()
window.title("Video Player")
window.geometry("740x2000")
window.config(bg="orange")
lbl1 = Label(window, text="HP Video Player", bg="orange red",
             fg="white", font="none 24 bold")
lbl1.config(anchor=CENTER)
lbl1.pack()

def openFile():
    file = askopenfile(mode='r', filetypes=[
                       ('Video Files', ["*.mp4"])])
    if file is not None:
        global filename
        filename = file.name
        global videoplayer
        videoplayer = TkinterVideo(master=window, scaled=True)
        videoplayer.load(r"{}".format(filename))
        videoplayer.pack(expand=True, fill="both")
        videoplayer.play()
def runfile():
    while True:
        cap = cv2.VideoCapture("New_4.mp4")

        while (cap.isOpened()):
            ret, frame = cap.read()
            frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
            h, w, _ = frame.shape
            x = cv2.resize(frame, (W, H))
            x = x/255.0
            x = x.astype(np.float32)
            x = np.expand_dims(x, axis=0)
            y = model.predict(x)[0]
            y = cv2.resize(y, (w, h))
            y = np.expand_dims(y, axis=-1)
            y = y > 0.5

            photo_mask = frame*y
            gray = cv2.cvtColor(photo_mask, cv2.COLOR_BGR2GRAY)
            blurred = cv2.GaussianBlur(gray,(301,301),0)
            ret3,thresholded_img = cv2.threshold(blurred,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            mapping = cv2.cvtColor(thresholded_img, cv2.COLOR_GRAY2RGB)
            np.unique(mapping)
            blurred_original_image = cv2.GaussianBlur(frame,(31,31),0)
            layered_image = np.where(mapping != (0,0,0), frame, blurred_original_image)
            cv2.imshow("Video Segmentation", layered_image) 
            c = cv2.waitKey(1)
            if c == 27:
             break  
        cap.release()
        cv2.destroyAllWindows()


openbtn= Button( window, text="Play original video", command=lambda:openFile()) 
openbtn.pack(side=TOP, pady=2)

playbtn= Button( window, text="Play Transformed Video", command=lambda:runfile())
playbtn.pack(side=TOP, pady=3)

window.mainloop()

2022-06-21 19:01:04.912761: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 396ms/step


: 

: 